In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
g = get("http://www.hsquizbowl.org/db/tournaments/4497/stats/round_robin/playerdetail/").text
soup = BeautifulSoup(g, "html5lib")

In [3]:
def player_from_table(table):
    df = pd.read_html(str(table), header=0, index_col=0)[0]
    # Recalculate
    del df["P/TU"]
    del df["P/N"]
    del df["G/N"]
    return df

In [4]:
name_fix = {
    "Han": "Yifan",
    "Darien": "Darren"  
}
last_name = {
    "Jasmine": "C",
    "Alison": "W",
    "Lui": "K"
}

players = {f"State College {l}": {} for l in "ABCDEF"}
for i in soup.find_all("h2"):
    try:
        player, team = map(str.strip, i.text.split(","))
    except:
        continue
    if not team.startswith("State College") or player == "Patrick":
        continue
    elif team == "State College D":
        first, last = player, last_name[player]
    else:
        first, last = player.split(" ")
        last = last[0]
        if first in name_fix:
            first = name_fix[first]
    players[team][f"{first} {last}"] = player_from_table(i.next_sibling.next_sibling)